### Data Preprocessing

In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
data = pd.read_csv('data/emb_data4.csv')
data = shuffle(data)

In [3]:
X1_raw = data['master_id']
X2_raw = data.drop(['master_id', 'assigne_state'], axis=1)

# Need to save
X2_mean = X2_raw.mean(axis = 0)
X2_std = X2_raw.std(axis = 0)

# Scaling
X2 = (X2_raw - X2_mean)/X2_std
X2 = X2.fillna(0)

y = data['assigne_state']

In [4]:
X1_lookup = [-1]
for i in tqdm(range(len(X1_raw))):
    if X1_raw[i] not in X1_lookup:
        X1_lookup.append(X1_raw[i])
# X1_lookup need to save

100%|██████████| 113184/113184 [00:04<00:00, 26572.74it/s]


In [5]:
X1 = pd.DataFrame(columns=['master_idx'], index = X1_raw.index)

for i in tqdm(range(len(X1_raw))):
    if X1_raw[i] in X1_lookup:
        X1['master_idx'][i] = X1_lookup.index(X1_raw[i])
    else:
        X1['master_idx'][i] = 0

100%|██████████| 113184/113184 [02:34<00:00, 733.89it/s]


In [6]:
X1_tst = X1[0:20000].as_matrix()
X2_tst = X2[0:20000].as_matrix()
y_tst = y[0:20000]#.as_matrix().reshape(-1,1)

X1_val = X1[20001:50000].as_matrix()
X2_val = X2[20001:50000].as_matrix()
y_val = y[20001:50000]#.as_matrix().reshape(-1,1)

X1_trn = X1[50001:].as_matrix()
X2_trn = X2[50001:].as_matrix()
y_trn = y[50001:]#.as_matrix().reshape(-1,1)

In [7]:
# X1_trn, X1_val, X2_trn, X2_val, y_trn, y_val = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

### Parallel Model

In [7]:
from keras.layers import Input, Embedding, merge
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers.merge import concatenate
from sklearn.metrics import roc_auc_score
n_masters = len(X1_lookup)
n_factors = 50

Using Theano backend.


In [8]:
master_in = Input(shape=(1,), dtype='int64', name='master_in')
m1 = Embedding(n_masters, n_factors, input_length=1)(master_in)
m2 = Flatten()(m1)
order_in = Input(shape=(12,), dtype='float32', name='order_in')

In [9]:
x = concatenate([m2, order_in])
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(200, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(100, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='sigmoid')(x)

In [10]:
model = Model([master_in, order_in], x)
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
master_in (InputLayer)           (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 50)         147450      master_in[0][0]                  
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 50)            0           embedding_1[0][0]                
____________________________________________________________________________________________________
order_in (InputLayer)            (None, 12)            0                                            
___________________________________________________________________________________________

In [11]:
class_weight = {1: 1.0, 0: y[y == 1].size / y[y == 0].size}

model.fit([X1_trn, X2_trn], y_trn, batch_size = 512, epochs=1000,
          validation_data=([X1_val, X2_val], y_val),
          class_weight = class_weight,
          callbacks = [EarlyStopping(monitor='val_acc', patience=30, verbose=1, min_delta=1e-4, mode='max'),
                       #ReduceLROnPlateau(monitor='val_acc', factor=0.3, patience=5, verbose=1, epsilon=1e-4, mode='max')
                        CSVLogger('data/learning_log.csv', separator=',', append=False),
                        ReduceLROnPlateau(monitor='val_acc', factor=0.3, patience=8, min_lr=0.00001, verbose = 1)
                           ])

Train on 63183 samples, validate on 29999 samples
Epoch 1/1000
63183/63183 [==============================] - 3s - loss: 0.3120 - acc: 0.5297 - val_loss: 0.7072 - val_acc: 0.4253
Epoch 2/1000
63183/63183 [==============================] - 3s - loss: 0.2791 - acc: 0.5510 - val_loss: 0.7189 - val_acc: 0.3824
Epoch 3/1000
63183/63183 [==============================] - 3s - loss: 0.2723 - acc: 0.5652 - val_loss: 0.7241 - val_acc: 0.3998
Epoch 4/1000
63183/63183 [==============================] - 3s - loss: 0.2685 - acc: 0.5801 - val_loss: 0.7078 - val_acc: 0.4750
Epoch 5/1000
63183/63183 [==============================] - 3s - loss: 0.2642 - acc: 0.5883 - val_loss: 0.7136 - val_acc: 0.4711
Epoch 6/1000
63183/63183 [==============================] - 3s - loss: 0.2610 - acc: 0.5988 - val_loss: 0.7048 - val_acc: 0.4877
Epoch 7/1000
63183/63183 [==============================] - 3s - loss: 0.2572 - acc: 0.6104 - val_loss: 0.6867 - val_acc: 0.5287
Epoch 8/1000
63183/63183 [=====================

In [12]:
y_pred = model.predict([X1_tst, X2_tst])
print(roc_auc_score(y_tst, y_pred))

0.753977008509


In [17]:
model.save_weights('data/model_01.h5')

## Dump preprocessing to pickle

In [19]:
from six.moves import cPickle as pickle

pickle_file = 'data/preprocessing.pickle'
try:
    f = open(pickle_file, 'wb')
    save = {
        'X1_lookup': X1_lookup,
        'X2_mean': X2_mean,
        'X2_std': X2_std
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

### Bins

In [39]:
y_pred2 = y_pred.reshape((20000,))

In [40]:
bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
group_names = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100']

categories = pd.cut(y_pred2, bins, labels=group_names)

df = pd.DataFrame(columns=['y_true', 'y_pred', 'bin'])
df['y_true'] = tst_y
df['y_pred'] = y_pred2
df['bin'] = categories

df.to_csv('bins.csv')

### Sequential Model

In [ ]:
n_masters = X.nunique()
n_factors = 100

In [ ]:
from keras.layers import Input, Embedding
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.models import Sequential, Model

In [ ]:
emb_model = Sequential([
    Embedding(n_masters, n_factors, input_length=1),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [ ]:
emb_model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.001), metrics=['accuracy'])
emb_model.summary()

In [ ]:
emb_model.fit(X_new['master_idx'], y, batch_size=256, epochs=100)